<a href="https://colab.research.google.com/github/WilliamJWen/Project42/blob/main/colab_notebooks/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project42: AI generated Image Detector**



# Data Preprocessing

In [17]:
# mount our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
%cd /content/drive/MyDrive/Project 42

/content/drive/MyDrive/Project 42


In [48]:
ls

 ChameleonDataset/  'Ethical Considerations.gdoc'  'Untitled Diagram.drawio'
 Chameleon.zip       processedChameleon/


### Preprocess Chameleon Dataset
- step 1: half of the real images (75-20-5)
- step 2: half of the real images (70-20-5) and same amount of the fake images (75-20-5)

In [33]:
#######################################
#### NO NEED TO RUN THIS PART AGAIN ####
########################################

import zipfile

zip_path = '/content/drive/MyDrive/Project 42/Chameleon.zip'
extract_to = '/content/destination_folder'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


In [36]:
#######################################
#### NO NEED TO RUN THIS PART AGAIN ####
########################################

import os
import random
import shutil
from math import floor

# -----------------------------
# 1. Define Paths
# -----------------------------
# Path to the original dataset containing:
#   - "1_real"
#   - "2_fake"
dataset_dir = "/content/destination_folder/Chameleon/test"  # CHANGE to your dataset location

# Output directories for each step
output_step1 = "/content/drive/MyDrive/Project 42/processedChameleon/step1"  # for step 1 (real only)
output_step2 = "/content/drive/MyDrive/Project 42/processedChameleon/step2"  # for step 2 (real + fake)

# Directories of real and fake images
real_dir = os.path.join(dataset_dir, "0_real")
fake_dir = os.path.join(dataset_dir, "1_fake")

# -----------------------------
# 2. Gather Image File Paths
# -----------------------------
valid_exts = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")

real_images = [os.path.join(real_dir, f) for f in os.listdir(real_dir)
               if f.lower().endswith(valid_exts)]
fake_images = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir)
               if f.lower().endswith(valid_exts)]

# Shuffle images to randomize splits
random.shuffle(real_images)
random.shuffle(fake_images)

print(f"Found {len(real_images)} real images.")
print(f"Found {len(fake_images)} fake images.")

# -----------------------------
# 3. Split Real Images into Two Halves
# -----------------------------
half_real_count = len(real_images) // 2
real_half_1 = real_images[:half_real_count]  # for step 1
real_half_2 = real_images[half_real_count:]  # for step 2

# -----------------------------
# 4. Define Splitting Functions
# -----------------------------
def split_75_20_5(image_list):
    """
    Splits a list of images into 75% train, 20% val, and 5% test.
    """
    total = len(image_list)
    train_end = floor(total * 0.75)
    val_end = floor(total * 0.95)  # 75% + 20% = 95%
    train = image_list[:train_end]
    val = image_list[train_end:val_end]
    test = image_list[val_end:]
    return train, val, test

def split_exact(image_list):
    """
    Splits a list into exactly 75% train, 20% val, 5% test.
    """
    total = len(image_list)
    train_end = floor(total * 0.75)
    val_end = floor(total * 0.95)
    train = image_list[:train_end]
    val = image_list[train_end:val_end]
    test = image_list[val_end:]
    return train, val, test

# -----------------------------
# 5. Create Step 1 Splits (Real Only)
# -----------------------------
train_real_1, val_real_1, test_real_1 = split_75_20_5(real_half_1)

# -----------------------------
# 6. Create Step 2 Splits (Balanced Real + Fake)
# -----------------------------
# Ensure that we have an equal number of fake images to match real_half_2
required_fake = len(real_half_2)
if required_fake > len(fake_images):
    raise ValueError("Not enough fake images to match the second half of real images.")

# Use as many fake images as needed
fake_for_step2 = fake_images[:required_fake]

# Shuffle before splitting each class separately
random.shuffle(real_half_2)
random.shuffle(fake_for_step2)

# Split each class individually
train_real_2, val_real_2, test_real_2 = split_exact(real_half_2)
train_fake,    val_fake,    test_fake    = split_exact(fake_for_step2)

# Combine corresponding splits to ensure balance in each subset
train_step2 = [(path, "real") for path in train_real_2] + [(path, "fake") for path in train_fake]
val_step2   = [(path, "real") for path in val_real_2]   + [(path, "fake") for path in val_fake]
test_step2  = [(path, "real") for path in test_real_2]  + [(path, "fake") for path in test_fake]

# Shuffle the combined splits
random.shuffle(train_step2)
random.shuffle(val_step2)
random.shuffle(test_step2)

# -----------------------------
# 7. Copy Files to Output Folders
# -----------------------------
def copy_files(file_list, output_dir, is_tuple=False):
    """
    Copies files to output_dir under subdirectories for each class.
    If is_tuple is True, each item is a (filepath, label) tuple.
    Otherwise, all files are assumed to be 'real'.
    """
    for item in file_list:
        if is_tuple:
            file_path, label = item
        else:
            file_path = item
            label = "real"
        dest_folder = os.path.join(output_dir, label)
        os.makedirs(dest_folder, exist_ok=True)
        file_name = os.path.basename(file_path)
        shutil.copy2(file_path, os.path.join(dest_folder, file_name))

# --- For Step 1 (real only) ---
for split, files in zip(["train", "val", "test"], [train_real_1, val_real_1, test_real_1]):
    out_dir = os.path.join(output_step1, split)
    os.makedirs(out_dir, exist_ok=True)
    copy_files(files, out_dir, is_tuple=False)

# --- For Step 2 (balanced real and fake) ---
for split, files in zip(["train", "val", "test"], [train_step2, val_step2, test_step2]):
    out_dir = os.path.join(output_step2, split)
    os.makedirs(out_dir, exist_ok=True)
    copy_files(files, out_dir, is_tuple=True)

print("Data successfully split and copied!")


Found 14863 real images.
Found 11170 fake images.
Data successfully split and copied!


Processed Data is at `/content/drive/MyDrive/Project\ 42/processedChameleon` now.

In [46]:
!ls -1 /content/Project42/dataset/processedChameleon/step2/val/real | wc -l


1486


In [51]:
import os
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

# set seed
torch.random.manual_seed(42)


# Define base path to processedChameleon
base_dir = "/content/drive/MyDrive/Project 42/processedChameleon"
step2_dir = os.path.join(base_dir, "step2")

# Define directories for each split
train_dir = os.path.join(step2_dir, "train")
val_dir   = os.path.join(step2_dir, "val")
test_dir  = os.path.join(step2_dir, "test")

# Define image transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load datasets using ImageFolder
train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset   = ImageFolder(root=val_dir, transform=transform)
test_dataset  = ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Step 2 Train classes:", train_dataset.classes)
print("Step 2 Train samples:", len(train_dataset))
print("Step 2 Validation samples:", len(val_dataset))
print("Step 2 Test samples:", len(test_dataset))


Step 2 Train classes: ['fake', 'real']
Step 2 Train samples: 11148
Step 2 Validation samples: 2972
Step 2 Test samples: 744


# Baseline Model

# Primary Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Baseline(nn.Module):
  def __init__(self):
    super(Baseline, self).__init__()

    # Hidden layer activation
    self.activation = F.relu

    # Average pooling
    self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2) # 2w x 2h -> w x h
    self.pool4 = nn.AvgPool2d(kernel_size=4, stride=4) # 4w x 4h -> w x h

    # Convolutional layers
    self.conv1 = nn.Conv2d(in_channels=3,
                           out_channels=64,
                           kernel_size=7,
                           padding=3,
                           stride=2)
    self.conv2 = nn.Conv2d(in_channels=64,
                           out_channels=64,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv3 = nn.Conv2d(in_channels=64,
                           out_channels=64,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv4 = nn.Conv2d(in_channels=64,
                           out_channels=128,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv5 = nn.Conv2d(in_channels=128,
                           out_channels=128,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv6 = nn.Conv2d(in_channels=128,
                           out_channels=256,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv7 = nn.Conv2d(in_channels=256,
                           out_channels=256,
                           kernel_size=3,
                           padding=1,
                           stride=1)
    self.conv8 = nn.Conv2d(in_channels=256,
                           out_channels=512,
                           kernel_size=3,
                           padding=1,
                           stride=2)
    self.conv9 = nn.Conv2d(in_channels=512,
                           out_channels=512,
                           kernel_size=3,
                           padding=1,
                           stride=1)

    # FC layer
    self.fc = nn.Linear(512 * 2 * 2, 1)


  def forward(self, x):
    # Layer 1
    x = self.conv1(x)                     # Output: 64x128x128
    x = self.pool2(x)                     # Output: 64x64x64

    # Layer 2
    skip = x.detach().clone()
    x = self.conv2(x)                     # Output: 64x64x64

    # Layer 3
    x = self.pool2(skip) + self.conv3(x)  # Output: 64x64x64

    # Layer 4
    skip = x.detach.clone()
    x = self.conv4(x)                     # Output: 128x32x32

    # Layer 5
    x = self.pool2(skip) + self.conv5(x)  # Output: 128x32x32

    # Layer 6
    skip = x.detach.clone()
    x = self.conv6(x)                     # Output: 256x16x16

    # Layer 7
    x = self.pool2(skip) + self.conv7(x)  # Output: 256x16x16

    # Layer 8
    skip = x.detach.clone()
    x = self.conv8(x)                     # Output: 512x8x8

    # Layer 9
    x = self.pool2(skip) + self.conv9(x)  # Output: 512x8x8
    x = self.pool4(x)                     # Output: 512x2x2

    # Layer 10
    x = x.view(-1, 512 * 2 * 2)
    x = self.fc(x)
    x = x.squeeze(1)

    return x


baseline_model = Baseline()
num_params = 0
for param in baseline_model.parameters():
    num_params += param.numel()
print("There are", num_params, "parameters in the baseline model")


There are 4732033 parameters in the baseline model


## Training

### Non-Baseline-Specific Functions

In [ ]:
import matplotlib.pyplot as plt
import torch


# Plot training curve from model path
def plot_training_curve(path):
  train_err = np.loadtxt("{}_train_err.csv".format(path))
  val_err = np.loadtxt("{}_val_err.csv".format(path))
  train_loss = np.loadtxt("{}_train_loss.csv".format(path))
  val_loss = np.loadtxt("{}_val_loss.csv".format(path))
  plt.title("Train vs Validation Error")
  n = len(train_err) # number of epochs
  plt.plot(range(1, n + 1), train_err, label="Train")
  plt.plot(range(1, n + 1), val_err, label="Validation")
  plt.xlabel("Epoch")
  plt.ylabel("Error")
  plt.legend(loc='best')
  plt.show()
  plt.title("Train vs Validation Loss")
  plt.plot(range(1, n + 1), train_loss, label="Train")
  plt.plot(range(1, n + 1), val_loss, label="Validation")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend(loc='best')
  plt.show()


# Given a tensor containing 2 possible values, normalize this to 0/1
def normalize_label(labels):
  max_val = torch.max(labels)
  min_val = torch.min(labels)
  norm_labels = (labels - min_val)/(max_val - min_val)
  return norm_labels


### Baseline-Specific Functions

In [ ]:
import numpy as np
import time
import torch.optim as optim
from torch.utils.data import DataLoader


# Get name of baseline model checkpoint
def get_baseline_name(epoch):
  path = "model_baseline_epoch{0}".format(epoch)
  return path


# Evaluate data from loader on net
def evaluate_baseline(net, loader):
  # Enable GPU usage
  if torch.cuda.is_available():
    net = net.cuda()

  total_loss = 0.0
  total_err = 0.0
  num_samples = 0

  criterion = nn.BinaryCrossEntropy()

  for i, data in enumerate(loader):
    imgs, labels = data
    labels = normalize_label(labels)
    num_samples += len(labels)

    # Enable GPU usage
    if torch.cuda.is_available():
      imgs = imgs.cuda()
      labels = labels.cuda()

    # Forward pass
    outputs = net(imgs)
    loss = criterion(outputs, labels)

    # Update loss, error
    total_loss += loss.item()
    for pred, label in zip(outputs, labels):
      if pred != label:
        total_err += 1

  err = float(total_err) / num_samples
  loss = float(total_loss) / len(loader)

  return err, loss


# Train baseline model on given datasets
def train_baseline(net, train_set, val_set):
  torch.manual_seed(1000)

  # Note: we want to train over 3000 iterations. For now, I assume a batch size
  # of ~6k, meaning there will be ~100 iterations/per epoch, so I am using 30
  # epochs.

  # Enable GPU usage
  if torch.cuda.is_available():
    net = net.cuda()

  # Hyperparameters
  batch_size = 64
  initial_learning_rate = 0.01
  num_epochs = 30

  # Criterion
  criterion = nn.BinaryCrossEntropy()

  # Optimizer
  betas = (0.9, 0.99)
  milestones = [10, 20]
  lr_factor = 0.1
  optimizer = optim.Adam(net.parameters(),
                         lr=initial_learning_rate,
                         betas=betas)
  learning_rate_schedule = optim.lr_scheduler.MultiStepLR(optimizer,
                                                          milestones=milestones,
                                                          gamma=lr_factor)

  # Arrays to record loss and error
  train_err = np.zeros(num_epochs)
  train_loss = np.zeros(num_epochs)
  val_err = np.zeros(num_epochs)
  val_loss = np.zeros(num_epochs)

  # Load data
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

  # Train
  start_time = time.time()
  for epoch in range(num_epochs):
    total_train_loss = 0.0
    total_train_err = 0.0
    num_samples = 0

    # Iterate through mini batches
    for i, data in enumerate(train_loader):
      imgs, labels = data
      num_samples += len(labels)

      # Enable GPU usage
      if torch.cuda.is_available():
        imgs = imgs.cuda()
        labels = labels.cuda()

      # Forward pass, backward pass, update
      optimizer.zero_grad()
      outputs = net(imgs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      # Update training loss, error
      total_loss += loss.item()
      for pred, label in zip(outputs, labels):
        if pred != label:
          total_err += 1

    # Calculate total loss, error
    train_err[epoch] = total_train_err / num_samples
    train_loss[epoch] = total_train_loss / len(train_loader)
    val_err[epoch], val_loss[epoch] = evaluate_baseline(net,
                                                        val_loader,
                                                        criterion)

    # Print loss, error
    print(("Epoch {}: Train err: {}, Train loss: {} | " +
           "Validation err: {}, Validation loss: {}").format(
              epoch + 1,
              train_err[epoch],
              train_loss[epoch],
              val_err[epoch],
              val_loss[epoch]))

    # Save the current model (checkpoint) to a file
    model_path = get_baseline_name(epoch + 1)
    torch.save(net.state_dict(), model_path)

    # Update LR scheduler
    learning_rate_schedule.step()

  # Finish up
  print('Finished Training')
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

  # Write the train/test loss/err into CSV file for plotting later
  np.savetxt("{}_train_err.csv".format(model_path), train_err)
  np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
  np.savetxt("{}_val_err.csv".format(model_path), val_err)
  np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
